Analysons les données

Les imports


In [1]:
import pandas as pd
import numpy as np
from numpy import random
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
import xgboost as xgb
from sklearn.metrics import accuracy_score,f1_score
from sklearn.model_selection import train_test_split
import imblearn

In [2]:
data=pd.read_csv('train.csv',sep=',')

In [3]:
data

,ID,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,196348,0,Cash loans,F,N,Y,2,225000.0,785398.5,33403.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,3.0
1,147976,0,Cash loans,M,N,Y,0,90000.0,592560.0,35937.0,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
2,52662,0,Revolving loans,F,N,Y,1,126000.0,202500.0,10125.0,...,0,0,0,0,0.0,0.0,0.0,1.0,0.0,4.0
3,101577,0,Cash loans,F,N,Y,0,157500.0,254700.0,18531.0,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
4,173078,0,Cash loans,F,Y,N,0,540000.0,1252363.5,47830.5,...,0,0,0,0,0.0,0.0,1.0,0.0,0.0,5.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
215253,119879,0,Cash loans,F,N,Y,0,121500.0,675000.0,32602.5,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
215254,103694,0,Cash loans,F,N,Y,0,202500.0,1503000.0,41463.0,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
215255,131932,0,Cash loans,M,Y,Y,0,450000.0,1125000.0,33025.5,...,0,0,0,0,0.0,0.0,0.0,2.0,0.0,3.0
215256,146867,0,Revolving loans,M,Y,Y,1,180000.0,270000.0,13500.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0


In [4]:
num_nan=data.isna().sum()
print(num_nan)

ID                                0
TARGET                            0
NAME_CONTRACT_TYPE                0
CODE_GENDER                       0
FLAG_OWN_CAR                      0
                              ...  
AMT_REQ_CREDIT_BUREAU_DAY     29072
AMT_REQ_CREDIT_BUREAU_WEEK    29072
AMT_REQ_CREDIT_BUREAU_MON     29072
AMT_REQ_CREDIT_BUREAU_QRT     29072
AMT_REQ_CREDIT_BUREAU_YEAR    29072
Length: 122, dtype: int64


One possible idea that comes to mind is to replace nan values with random unique out of possible range values for numeric data and as for categorical data ,we create a new class called "unknown",because missing information is an information itself

extract numeric and categorical data

In [5]:
categorical=list(data.select_dtypes(exclude=["number","bool_"]).columns.values)
numeric=list(data.drop(columns=["TARGET"])._get_numeric_data().columns)

In [6]:
categorical

['NAME_CONTRACT_TYPE',
 'CODE_GENDER',
 'FLAG_OWN_CAR',
 'FLAG_OWN_REALTY',
 'NAME_TYPE_SUITE',
 'NAME_INCOME_TYPE',
 'NAME_EDUCATION_TYPE',
 'NAME_FAMILY_STATUS',
 'NAME_HOUSING_TYPE',
 'OCCUPATION_TYPE',
 'WEEKDAY_APPR_PROCESS_START',
 'ORGANIZATION_TYPE',
 'FONDKAPREMONT_MODE',
 'HOUSETYPE_MODE',
 'WALLSMATERIAL_MODE',
 'EMERGENCYSTATE_MODE']

In [7]:
data[categorical]=data[categorical].fillna("unknown")

In [8]:

for col in numeric:
    min=float(np.abs(data[col].min()))
    alea=random.randint(100)*-1+min
    data[col]=data[col].fillna(alea)


In [9]:
np.unique(data.isna().sum())

array([0], dtype=int64)

As first step ,before deciding with which model will carry on , we need to study the nature of relationship between data

In [10]:
cols=list(data.drop(columns=["TARGET"]).columns.values)
cols

['ID',
 'NAME_CONTRACT_TYPE',
 'CODE_GENDER',
 'FLAG_OWN_CAR',
 'FLAG_OWN_REALTY',
 'CNT_CHILDREN',
 'AMT_INCOME_TOTAL',
 'AMT_CREDIT',
 'AMT_ANNUITY',
 'AMT_GOODS_PRICE',
 'NAME_TYPE_SUITE',
 'NAME_INCOME_TYPE',
 'NAME_EDUCATION_TYPE',
 'NAME_FAMILY_STATUS',
 'NAME_HOUSING_TYPE',
 'REGION_POPULATION_RELATIVE',
 'DAYS_BIRTH',
 'DAYS_EMPLOYED',
 'DAYS_REGISTRATION',
 'DAYS_ID_PUBLISH',
 'OWN_CAR_AGE',
 'FLAG_MOBIL',
 'FLAG_EMP_PHONE',
 'FLAG_WORK_PHONE',
 'FLAG_CONT_MOBILE',
 'FLAG_PHONE',
 'FLAG_EMAIL',
 'OCCUPATION_TYPE',
 'CNT_FAM_MEMBERS',
 'REGION_RATING_CLIENT',
 'REGION_RATING_CLIENT_W_CITY',
 'WEEKDAY_APPR_PROCESS_START',
 'HOUR_APPR_PROCESS_START',
 'REG_REGION_NOT_LIVE_REGION',
 'REG_REGION_NOT_WORK_REGION',
 'LIVE_REGION_NOT_WORK_REGION',
 'REG_CITY_NOT_LIVE_CITY',
 'REG_CITY_NOT_WORK_CITY',
 'LIVE_CITY_NOT_WORK_CITY',
 'ORGANIZATION_TYPE',
 'EXT_SOURCE_1',
 'EXT_SOURCE_2',
 'EXT_SOURCE_3',
 'APARTMENTS_AVG',
 'BASEMENTAREA_AVG',
 'YEARS_BEGINEXPLUATATION_AVG',
 'YEARS_BUILD_

ENCODING

In [11]:
features_encoded=pd.get_dummies(data,columns=categorical)
features_encoded

,ID,TARGET,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,...,WALLSMATERIAL_MODE_Mixed,WALLSMATERIAL_MODE_Monolithic,WALLSMATERIAL_MODE_Others,WALLSMATERIAL_MODE_Panel,"WALLSMATERIAL_MODE_Stone, brick",WALLSMATERIAL_MODE_Wooden,WALLSMATERIAL_MODE_unknown,EMERGENCYSTATE_MODE_No,EMERGENCYSTATE_MODE_Yes,EMERGENCYSTATE_MODE_unknown
0,196348,0,2,225000.0,785398.5,33403.5,702000.0,0.046220,-13629,-514,...,False,False,False,False,False,False,True,False,False,True
1,147976,0,0,90000.0,592560.0,35937.0,450000.0,0.025164,-11983,-1863,...,False,False,False,True,False,False,False,True,False,False
2,52662,0,1,126000.0,202500.0,10125.0,202500.0,0.025164,-15182,-2033,...,False,False,False,False,False,False,True,False,False,True
3,101577,0,0,157500.0,254700.0,18531.0,225000.0,0.006207,-13214,-232,...,False,False,False,True,False,False,False,True,False,False
4,173078,0,0,540000.0,1252363.5,47830.5,1152000.0,0.072508,-19745,-713,...,False,False,False,False,True,False,False,True,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
215253,119879,0,0,121500.0,675000.0,32602.5,675000.0,0.006305,-14481,-1124,...,False,False,False,False,False,False,True,False,False,True
215254,103694,0,0,202500.0,1503000.0,41463.0,1503000.0,0.004960,-18097,-1557,...,False,False,False,False,False,False,True,False,False,True
215255,131932,0,0,450000.0,1125000.0,33025.5,1125000.0,0.014520,-14309,-5929,...,False,False,False,True,False,False,False,True,False,False
215256,146867,0,1,180000.0,270000.0,13500.0,270000.0,0.010006,-12706,-1868,...,False,False,False,False,True,False,False,True,False,False


In [12]:
#features_encoded=features_encoded.drop(columns=['CODE_GENDER_XNA', 'NAME_FAMILY_STATUS_Unknown'])

In [13]:
correlation_matrix = features_encoded.corr()
# Sélectionnez seulement la colonne de la variable target pour voir les corrélations avec toutes les autres variables
target_correlation = correlation_matrix['TARGET']
print(target_correlation)

ID                            -0.002793
TARGET                         1.000000
CNT_CHILDREN                   0.020044
AMT_INCOME_TOTAL              -0.001546
AMT_CREDIT                    -0.030230
                                 ...   
WALLSMATERIAL_MODE_Wooden      0.007796
WALLSMATERIAL_MODE_unknown     0.039865
EMERGENCYSTATE_MODE_No        -0.042609
EMERGENCYSTATE_MODE_Yes        0.005508
EMERGENCYSTATE_MODE_unknown    0.041690
Name: TARGET, Length: 252, dtype: float64


with these weak correlation ,it is preferable to avoid models that assume linearity with the target variable

Une technique très populaire dans la prédiction de default est le boosting

In [14]:
features_encoded

,ID,TARGET,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,...,WALLSMATERIAL_MODE_Mixed,WALLSMATERIAL_MODE_Monolithic,WALLSMATERIAL_MODE_Others,WALLSMATERIAL_MODE_Panel,"WALLSMATERIAL_MODE_Stone, brick",WALLSMATERIAL_MODE_Wooden,WALLSMATERIAL_MODE_unknown,EMERGENCYSTATE_MODE_No,EMERGENCYSTATE_MODE_Yes,EMERGENCYSTATE_MODE_unknown
0,196348,0,2,225000.0,785398.5,33403.5,702000.0,0.046220,-13629,-514,...,False,False,False,False,False,False,True,False,False,True
1,147976,0,0,90000.0,592560.0,35937.0,450000.0,0.025164,-11983,-1863,...,False,False,False,True,False,False,False,True,False,False
2,52662,0,1,126000.0,202500.0,10125.0,202500.0,0.025164,-15182,-2033,...,False,False,False,False,False,False,True,False,False,True
3,101577,0,0,157500.0,254700.0,18531.0,225000.0,0.006207,-13214,-232,...,False,False,False,True,False,False,False,True,False,False
4,173078,0,0,540000.0,1252363.5,47830.5,1152000.0,0.072508,-19745,-713,...,False,False,False,False,True,False,False,True,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
215253,119879,0,0,121500.0,675000.0,32602.5,675000.0,0.006305,-14481,-1124,...,False,False,False,False,False,False,True,False,False,True
215254,103694,0,0,202500.0,1503000.0,41463.0,1503000.0,0.004960,-18097,-1557,...,False,False,False,False,False,False,True,False,False,True
215255,131932,0,0,450000.0,1125000.0,33025.5,1125000.0,0.014520,-14309,-5929,...,False,False,False,True,False,False,False,True,False,False
215256,146867,0,1,180000.0,270000.0,13500.0,270000.0,0.010006,-12706,-1868,...,False,False,False,False,True,False,False,True,False,False


In [68]:
from sklearn.ensemble import RandomForestClassifier,GradientBoostingClassifier
X=features_encoded.drop(columns=["TARGET"])
y=features_encoded['TARGET']

In [16]:
#feature importance

gbm = GradientBoostingClassifier(n_estimators=100, learning_rate=1.0, max_depth=1, random_state=42)
gbm.fit(X, y)


# Feature importance
print("Feature Importances:", gbm.feature_importances_)

Feature Importances: [0.0084312  0.00114375 0.         0.02106991 0.01195828 0.02573656
 0.         0.04629575 0.01881074 0.00089614 0.00677226 0.00172045
 0.         0.         0.0016509  0.         0.         0.
 0.00086486 0.         0.0053775  0.         0.         0.
 0.         0.00131007 0.         0.         0.04406843 0.39285349
 0.29080217 0.         0.         0.00083615 0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.00742467 0.         0.00790778 0.         0.00089849
 0.00312569 0.         0.01950953 0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.00118252 0.         0.0010

In [69]:
importances=gbm.feature_importances_
print(gbm.feature_importances_.max())

0.392853490056421


In [70]:
threshold = np.mean(importances)

# Find the indices of important features
important_indices = np.where(importances >= threshold)[0]

In [71]:
print(threshold)
len(important_indices)

0.00398406374501992


18

In [72]:
len(X.columns)

251

In [73]:
X

,ID,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,...,WALLSMATERIAL_MODE_Mixed,WALLSMATERIAL_MODE_Monolithic,WALLSMATERIAL_MODE_Others,WALLSMATERIAL_MODE_Panel,"WALLSMATERIAL_MODE_Stone, brick",WALLSMATERIAL_MODE_Wooden,WALLSMATERIAL_MODE_unknown,EMERGENCYSTATE_MODE_No,EMERGENCYSTATE_MODE_Yes,EMERGENCYSTATE_MODE_unknown
0,196348,2,225000.0,785398.5,33403.5,702000.0,0.046220,-13629,-514,-7712.0,...,False,False,False,False,False,False,True,False,False,True
1,147976,0,90000.0,592560.0,35937.0,450000.0,0.025164,-11983,-1863,-11266.0,...,False,False,False,True,False,False,False,True,False,False
2,52662,1,126000.0,202500.0,10125.0,202500.0,0.025164,-15182,-2033,-4189.0,...,False,False,False,False,False,False,True,False,False,True
3,101577,0,157500.0,254700.0,18531.0,225000.0,0.006207,-13214,-232,-440.0,...,False,False,False,True,False,False,False,True,False,False
4,173078,0,540000.0,1252363.5,47830.5,1152000.0,0.072508,-19745,-713,-11317.0,...,False,False,False,False,True,False,False,True,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
215253,119879,0,121500.0,675000.0,32602.5,675000.0,0.006305,-14481,-1124,-3813.0,...,False,False,False,False,False,False,True,False,False,True
215254,103694,0,202500.0,1503000.0,41463.0,1503000.0,0.004960,-18097,-1557,-8992.0,...,False,False,False,False,False,False,True,False,False,True
215255,131932,0,450000.0,1125000.0,33025.5,1125000.0,0.014520,-14309,-5929,-5038.0,...,False,False,False,True,False,False,False,True,False,False
215256,146867,1,180000.0,270000.0,13500.0,270000.0,0.010006,-12706,-1868,-923.0,...,False,False,False,False,True,False,False,True,False,False


In [74]:
# Filter your features
#X = X[:, important_indices]

important_indices = [i for i, imp in enumerate(importances) if imp >= threshold]

# Get the names of important features using these indices
important_features = X.columns[important_indices]

In [75]:
important_features

Index(['ID', 'AMT_CREDIT', 'AMT_ANNUITY', 'AMT_GOODS_PRICE', 'DAYS_BIRTH',
       'DAYS_EMPLOYED', 'DAYS_ID_PUBLISH', 'REGION_RATING_CLIENT_W_CITY',
       'EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3', 'TOTALAREA_MODE',
       'DEF_30_CNT_SOCIAL_CIRCLE', 'FLAG_DOCUMENT_3', 'CODE_GENDER_M',
       'FLAG_OWN_CAR_N', 'NAME_INCOME_TYPE_Unemployed',
       'NAME_EDUCATION_TYPE_Higher education'],
      dtype='object')

In [76]:
X= X[important_features]


In [77]:
len(X.columns)

18

In [78]:
#add validation data
from imblearn.over_sampling import SMOTE
scaler=StandardScaler()
#X=features_encoded.drop(columns=["TARGET"])
#check dataset balance

print (np.sum(features_encoded['TARGET']==1))
print (np.sum(features_encoded['TARGET']==0))

y=features_encoded['TARGET']
print(X)

x_train,x_val,y_train,y_val=train_test_split(X,y,test_size=0.3,random_state=42)
sm = SMOTE(sampling_strategy=0.2, random_state=42,k_neighbors=10)
X_res, y_res = sm.fit_resample(x_train, y_train)
scaler=StandardScaler()
x_train_res=scaler.fit_transform(X_res)
x_val_res=scaler.fit_transform(x_val)



17378
197880
            ID  AMT_CREDIT  AMT_ANNUITY  AMT_GOODS_PRICE  DAYS_BIRTH  \
0       196348    785398.5      33403.5         702000.0      -13629   
1       147976    592560.0      35937.0         450000.0      -11983   
2        52662    202500.0      10125.0         202500.0      -15182   
3       101577    254700.0      18531.0         225000.0      -13214   
4       173078   1252363.5      47830.5        1152000.0      -19745   
...        ...         ...          ...              ...         ...   
215253  119879    675000.0      32602.5         675000.0      -14481   
215254  103694   1503000.0      41463.0        1503000.0      -18097   
215255  131932   1125000.0      33025.5        1125000.0      -14309   
215256  146867    270000.0      13500.0         270000.0      -12706   
215257  121958    729792.0      35239.5         630000.0      -13176   

        DAYS_EMPLOYED  DAYS_ID_PUBLISH  REGION_RATING_CLIENT_W_CITY  \
0                -514            -2405             

In [79]:

def f1_eval(y_true, y_pred):
    # XGBClassifier outputs probabilities by default. We need to convert these to binary labels.
    y_pred_binary = (y_pred > 0.5).astype(int)
    return 'f1', f1_score(y_true, y_pred_binary)

In [80]:
from sklearn.model_selection import RandomizedSearchCV
model=xgb.XGBClassifier(objective='binary:logistic',eval_metric='logloss',use_label_encoder=False,early_stopping_rounds=10)
model.fit(x_train_res,y_res,eval_set=[(x_val_res,y_val)])

[0]	validation_0-logloss:0.42264
[1]	validation_0-logloss:0.49323
[2]	validation_0-logloss:0.59773
[3]	validation_0-logloss:0.73345
[4]	validation_0-logloss:0.85065
[5]	validation_0-logloss:0.95797
[6]	validation_0-logloss:0.95917
[7]	validation_0-logloss:1.02093
[8]	validation_0-logloss:1.03210
[9]	validation_0-logloss:1.10925


c:\Users\asmak\anaconda3\envs\python310\lib\site-packages\xgboost\core.py:158: UserWarning: [01:59:33] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0c55ff5f71b100e98-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=10,
              enable_categorical=False, eval_metric='logloss',
              feature_types=None, gamma=None, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=None, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=None,
              max_leaves=None, min_child_weight=None, missing=nan,
              monotone_constraints=None, multi_strategy=None, n_estimators=None,
              n_jobs=None, num_parallel_tree=None, random_state=None, ...)

In [81]:


# Predict probabilities
y_pred_prob = model.predict_proba(x_val)[:, 1]

# Convert probabilities to binary labels
y_pred = (y_pred_prob > 0.5).astype(int)

# Calculate F1 score
f1 = f1_score(y_val, y_pred)
print("F1 Score:", f1)

F1 Score: 0.19194107452339687


In [82]:
#randomforestclassifier use
from sklearn.metrics import f1_score, classification_report

# Initialize and train a Random Forest classifier
rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42)
rf_classifier.fit(x_train_res, y_res)



RandomForestClassifier(random_state=42)

In [83]:
# Predict and evaluate
y_pred_rf = rf_classifier.predict(x_val_res)
print("Random Forest F1 Score:", f1_score(y_val, y_pred_rf))
print(classification_report(y_val, y_pred_rf))

Random Forest F1 Score: 0.15822233689166962
              precision    recall  f1-score   support

           0       0.96      0.13      0.22     59355
           1       0.09      0.94      0.16      5223

    accuracy                           0.19     64578
   macro avg       0.52      0.53      0.19     64578
weighted avg       0.89      0.19      0.22     64578



In [84]:
#neural network
from keras.models import Sequential
from keras.layers import Dense

# Create a simple neural network
model = Sequential()
model.add(Dense(12, input_dim=x_train_res.shape[1], activation='relu'))
model.add(Dense(8, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['f1_score'])

# Train the model
model.fit(x_train_res, y_res, epochs=10, batch_size=10)

# Evaluate the model
_, accuracy = model.evaluate(x_val_res, y_val)
#print("Neural Network Accuracy:", accuracy)

Epoch 1/10


c:\Users\asmak\anaconda3\envs\python310\lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


16623/16623 ━━━━━━━━━━━━━━━━━━━━ 16s 918us/step - f1_score: 0.2867 - loss: 0.4125
Epoch 2/10
16623/16623 ━━━━━━━━━━━━━━━━━━━━ 27s 2ms/step - f1_score: 0.2861 - loss: 0.3455
Epoch 3/10
16623/16623 ━━━━━━━━━━━━━━━━━━━━ 30s 2ms/step - f1_score: 0.2831 - loss: 0.3175
Epoch 4/10
16623/16623 ━━━━━━━━━━━━━━━━━━━━ 15s 893us/step - f1_score: 0.2852 - loss: 0.3057
Epoch 5/10
16623/16623 ━━━━━━━━━━━━━━━━━━━━ 28s 2ms/step - f1_score: 0.2859 - loss: 0.2993
Epoch 6/10
16623/16623 ━━━━━━━━━━━━━━━━━━━━ 34s 1ms/step - f1_score: 0.2855 - loss: 0.2940
Epoch 7/10
16623/16623 ━━━━━━━━━━━━━━━━━━━━ 32s 2ms/step - f1_score: 0.2850 - loss: 0.2909
Epoch 8/10
16623/16623 ━━━━━━━━━━━━━━━━━━━━ 44s 2ms/step - f1_score: 0.2848 - loss: 0.2910
Epoch 9/10
16623/16623 ━━━━━━━━━━━━━━━━━━━━ 36s 2ms/step - f1_score: 0.2857 - loss: 0.2892
Epoch 10/10
16623/16623 ━━━━━━━━━━━━━━━━━━━━ 27s 1ms/step - f1_score: 0.2871 - loss: 0.2889
2019/2019 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - f1_score: 0.1463 - loss: 0.2841


test data

In [85]:
test=pd.read_csv('test.csv',sep=',')
test.isna().sum()

ID                                0
NAME_CONTRACT_TYPE                0
CODE_GENDER                       0
FLAG_OWN_CAR                      0
FLAG_OWN_REALTY                   0
                              ...  
AMT_REQ_CREDIT_BUREAU_DAY     12447
AMT_REQ_CREDIT_BUREAU_WEEK    12447
AMT_REQ_CREDIT_BUREAU_MON     12447
AMT_REQ_CREDIT_BUREAU_QRT     12447
AMT_REQ_CREDIT_BUREAU_YEAR    12447
Length: 121, dtype: int64

In [86]:
data

,ID,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,196348,0,Cash loans,F,N,Y,2,225000.0,785398.5,33403.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,3.0
1,147976,0,Cash loans,M,N,Y,0,90000.0,592560.0,35937.0,...,0,0,0,0,-55.0,-43.0,-94.0,-72.0,-9.0,-5.0
2,52662,0,Revolving loans,F,N,Y,1,126000.0,202500.0,10125.0,...,0,0,0,0,0.0,0.0,0.0,1.0,0.0,4.0
3,101577,0,Cash loans,F,N,Y,0,157500.0,254700.0,18531.0,...,0,0,0,0,-55.0,-43.0,-94.0,-72.0,-9.0,-5.0
4,173078,0,Cash loans,F,Y,N,0,540000.0,1252363.5,47830.5,...,0,0,0,0,0.0,0.0,1.0,0.0,0.0,5.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
215253,119879,0,Cash loans,F,N,Y,0,121500.0,675000.0,32602.5,...,0,0,0,0,-55.0,-43.0,-94.0,-72.0,-9.0,-5.0
215254,103694,0,Cash loans,F,N,Y,0,202500.0,1503000.0,41463.0,...,0,0,0,0,-55.0,-43.0,-94.0,-72.0,-9.0,-5.0
215255,131932,0,Cash loans,M,Y,Y,0,450000.0,1125000.0,33025.5,...,0,0,0,0,0.0,0.0,0.0,2.0,0.0,3.0
215256,146867,0,Revolving loans,M,Y,Y,1,180000.0,270000.0,13500.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0


In [87]:
test

,ID,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,...,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,37408,Cash loans,M,N,Y,0,90000.0,247500.0,19971.0,247500.0,...,0,0,0,0,0.0,0.0,0.0,0.0,1.0,1.0
1,75132,Cash loans,M,N,N,1,121500.0,247275.0,19134.0,225000.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,4.0
2,21513,Cash loans,F,N,Y,0,112500.0,679500.0,26946.0,679500.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,2.0
3,58291,Revolving loans,F,N,Y,0,135000.0,405000.0,20250.0,405000.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
4,39033,Cash loans,F,N,Y,0,121500.0,604413.0,21838.5,459000.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
92248,6265,Cash loans,M,N,N,0,225000.0,239850.0,25960.5,225000.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
92249,54886,Revolving loans,M,N,Y,0,180000.0,247500.0,12375.0,247500.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,4.0
92250,76820,Cash loans,F,Y,Y,0,135000.0,490536.0,31936.5,405000.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,3.0
92251,860,Cash loans,M,Y,Y,0,270000.0,1247472.0,52983.0,1147500.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0


In [88]:
features_encoded

,ID,TARGET,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,...,WALLSMATERIAL_MODE_Mixed,WALLSMATERIAL_MODE_Monolithic,WALLSMATERIAL_MODE_Others,WALLSMATERIAL_MODE_Panel,"WALLSMATERIAL_MODE_Stone, brick",WALLSMATERIAL_MODE_Wooden,WALLSMATERIAL_MODE_unknown,EMERGENCYSTATE_MODE_No,EMERGENCYSTATE_MODE_Yes,EMERGENCYSTATE_MODE_unknown
0,196348,0,2,225000.0,785398.5,33403.5,702000.0,0.046220,-13629,-514,...,False,False,False,False,False,False,True,False,False,True
1,147976,0,0,90000.0,592560.0,35937.0,450000.0,0.025164,-11983,-1863,...,False,False,False,True,False,False,False,True,False,False
2,52662,0,1,126000.0,202500.0,10125.0,202500.0,0.025164,-15182,-2033,...,False,False,False,False,False,False,True,False,False,True
3,101577,0,0,157500.0,254700.0,18531.0,225000.0,0.006207,-13214,-232,...,False,False,False,True,False,False,False,True,False,False
4,173078,0,0,540000.0,1252363.5,47830.5,1152000.0,0.072508,-19745,-713,...,False,False,False,False,True,False,False,True,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
215253,119879,0,0,121500.0,675000.0,32602.5,675000.0,0.006305,-14481,-1124,...,False,False,False,False,False,False,True,False,False,True
215254,103694,0,0,202500.0,1503000.0,41463.0,1503000.0,0.004960,-18097,-1557,...,False,False,False,False,False,False,True,False,False,True
215255,131932,0,0,450000.0,1125000.0,33025.5,1125000.0,0.014520,-14309,-5929,...,False,False,False,True,False,False,False,True,False,False
215256,146867,0,1,180000.0,270000.0,13500.0,270000.0,0.010006,-12706,-1868,...,False,False,False,False,True,False,False,True,False,False


In [89]:

categorical_test=list(test.select_dtypes(exclude=["number","bool_"]).columns.values)
numeric_test=list(test._get_numeric_data().columns)
test[categorical_test]=test[categorical_test].fillna("unknown")
for col in numeric_test:
    min=float(np.abs(test[col].min()))
    alea=random.randint(100)*-1+min
    test[col]=test[col].fillna(alea)
cols=list(test.columns.values)

features_encoded_test=pd.get_dummies(test,columns=categorical_test)




In [90]:
len(categorical)

16

In [91]:
len(categorical_test)

16

In [92]:
len(numeric)

105

In [93]:
len(numeric_test)

105

In [94]:
features_encoded_test

,ID,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,...,WALLSMATERIAL_MODE_Mixed,WALLSMATERIAL_MODE_Monolithic,WALLSMATERIAL_MODE_Others,WALLSMATERIAL_MODE_Panel,"WALLSMATERIAL_MODE_Stone, brick",WALLSMATERIAL_MODE_Wooden,WALLSMATERIAL_MODE_unknown,EMERGENCYSTATE_MODE_No,EMERGENCYSTATE_MODE_Yes,EMERGENCYSTATE_MODE_unknown
0,37408,0,90000.0,247500.0,19971.0,247500.0,0.028663,-16121,-582,-6226.0,...,False,False,False,False,True,False,False,True,False,False
1,75132,1,121500.0,247275.0,19134.0,225000.0,0.015221,-12586,-1133,-5319.0,...,False,False,False,False,False,False,True,False,False,True
2,21513,0,112500.0,679500.0,26946.0,679500.0,0.008625,-13473,-4466,-6354.0,...,False,False,False,False,False,False,True,False,False,True
3,58291,0,135000.0,405000.0,20250.0,405000.0,0.032561,-8468,-824,-8395.0,...,False,False,False,True,False,False,False,True,False,False
4,39033,0,121500.0,604413.0,21838.5,459000.0,0.035792,-22479,365243,-8837.0,...,False,False,False,False,False,False,True,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
92248,6265,0,225000.0,239850.0,25960.5,225000.0,0.011703,-14120,-3751,-8112.0,...,False,False,False,False,True,False,False,True,False,False
92249,54886,0,180000.0,247500.0,12375.0,247500.0,0.019101,-13086,-2148,-3674.0,...,False,False,False,False,False,False,True,False,False,True
92250,76820,0,135000.0,490536.0,31936.5,405000.0,0.018029,-10403,-2403,-1124.0,...,False,False,False,False,False,False,True,False,False,True
92251,860,0,270000.0,1247472.0,52983.0,1147500.0,0.006852,-18571,-7045,-4269.0,...,False,False,False,False,False,False,True,False,False,True


In [95]:
l1=list(features_encoded.columns.values)
l2=list(features_encoded_test.columns.values)
diff=set(l1)-set(l2)
diff

{'CODE_GENDER_XNA', 'NAME_FAMILY_STATUS_Unknown', 'TARGET'}

In [96]:
test['CODE_GENDER']

0        M
1        M
2        F
3        F
4        F
        ..
92248    M
92249    M
92250    F
92251    M
92252    F
Name: CODE_GENDER, Length: 92253, dtype: object

In [97]:
features_encoded_test['CODE_GENDER_XNA']=False

In [98]:
features_encoded_test['CODE_GENDER_XNA']

0        False
1        False
2        False
3        False
4        False
         ...  
92248    False
92249    False
92250    False
92251    False
92252    False
Name: CODE_GENDER_XNA, Length: 92253, dtype: bool

In [99]:
features_encoded_test['NAME_FAMILY_STATUS_Unknown']=False

In [100]:
features_encoded_test['NAME_FAMILY_STATUS_Unknown']

0        False
1        False
2        False
3        False
4        False
         ...  
92248    False
92249    False
92250    False
92251    False
92252    False
Name: NAME_FAMILY_STATUS_Unknown, Length: 92253, dtype: bool

In [101]:
features_encoded[['NAME_FAMILY_STATUS_Unknown']]

,NAME_FAMILY_STATUS_Unknown
0,False
1,False
2,False
3,False
4,False
...,...
215253,False
215254,False
215255,False
215256,False


In [102]:
len(test.columns)

121

In [103]:
len(data.columns)

122

In [104]:

l1=list(features_encoded.columns.values)
l2=list(features_encoded_test.columns.values)
diff=set(l1)-set(l2)
diff



{'TARGET'}

In [105]:
selected=features_encoded_test[important_features]

In [106]:
scaler=StandardScaler()
X=scaler.fit_transform(selected)

In [107]:
y_pred=model.predict(X)
y_pred

2883/2883 ━━━━━━━━━━━━━━━━━━━━ 2s 538us/step


array([[0.14627635],
       [0.4430309 ],
       [0.1668544 ],
       ...,
       [0.34143794],
       [0.07996942],
       [0.09158143]], dtype=float32)

In [108]:
y_pred = (y_pred > 0.5).astype(int)

In [109]:
np.unique(y_pred)

array([0, 1])

In [110]:
y_pred_flat = y_pred.ravel()
y_pred_flat

array([0, 0, 0, ..., 0, 0, 0])

In [111]:
predictions=list(y_pred_flat)
id=list(test['ID'].values)

In [112]:
d={'ID':id,'TARGET':predictions}
submission=pd.DataFrame(data=d)



In [113]:
submission.to_csv('TEAMASMA.csv',index=None)

no feature engineering ==> f1 score 0.01748 on test data (aka F1 Score: 0.030579899310087637
)